In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision import utils
from sklearn.model_selection import train_test_split
from PIL import Image
import glob
import random
import time
import copy
import torch.nn as nn

In [11]:
test1 = np.load("/home/xiangjianhou/hc701-fed/preprocessed/aptos/train/0a4e1a29ffff.npy", allow_pickle=True).item()

In [12]:
(test1['image'] / test1['image'].max()).astype(np.float32)

dtype('float64')

In [14]:
test1['label']

AttributeError: 'int' object has no attribute 'dtype'